In [3]:
# os - operating system
import os
# io - input/output
import io
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
path = 'emails/'
for root, folder, fileName in os.walk(path):
#     print(root,folder,fileName)
    pass

In [5]:
fileName[0]

'00001.7848dde101aa985090474a91ec93fcf0'

In [6]:
for root, folder, fileName in os.walk(path):
    for file in fileName:
#         print(os.path.join(root,file))
        path = os.path.join(root,file)

In [7]:
path

'emails/spam\\00500.85b72f09f6778a085dc8b6821965a76f'

In [8]:
file_1 = 'emails/ham/00002.9c4069e25e1ef370c078db7ee85ff9ac'

In [9]:
inBody = False
lines = []
f = io.open(file_1,'r')
for line in f:
#     print(line)
    if inBody == True:
        lines.append(line)
#         print(lines)
    elif line == "\n":
        inBody = True
f.close()

In [10]:
print(lines)

['Martin A posted:\n', 'Tassos Papadopoulos, the Greek sculptor behind the plan, judged that the\n', ' limestone of Mount Kerdylio, 70 miles east of Salonika and not far from the\n', ' Mount Athos monastic community, was ideal for the patriotic sculpture. \n', ' \n', " As well as Alexander's granite features, 240 ft high and 170 ft wide, a\n", ' museum, a restored amphitheatre and car park for admiring crowds are\n', 'planned\n', '---------------------\n', 'So is this mountain limestone or granite?\n', "If it's limestone, it'll weather pretty fast.\n", '\n', '------------------------ Yahoo! Groups Sponsor ---------------------~-->\n', '4 DVDs Free +s&p Join Now\n', 'http://us.click.yahoo.com/pt6YBB/NXiEAA/mG3HAA/7gSolB/TM\n', '---------------------------------------------------------------------~->\n', '\n', 'To unsubscribe from this group, send an email to:\n', 'forteana-unsubscribe@egroups.com\n', '\n', ' \n', '\n', 'Your use of Yahoo! Groups is subject to http://docs.yahoo.com/info/

In [32]:
def readFile(path):
    for root, folder, files in os.walk(path):
        for fileName in files:
            file = os.path.join(root,fileName)
            
            inBody = False
            lines = []
            f = io.open(file,'r',encoding='latin1')
            for line in f:
            #     print(line)
                if inBody == True:
                    lines.append(line)
            #         print(lines)
                elif line == "\n":
                    inBody = True
            f.close()
            message = "\n".join(lines)
            yield message, path

In [33]:
def buildDataFrame(path, classification):
    
    row = []
    index = []
    
    for message, filename in readFile(path):
        row.append({'message':message, "class":classification})
        index.append(filename)
        
    return pd.DataFrame(data=row, index=index)

In [34]:
dataset = pd.DataFrame({'message':[], 'class':[]})

dataset = dataset.append(buildDataFrame('emails/spam', 'spam'))
dataset = dataset.append(buildDataFrame('emails/ham', 'ham'))

In [35]:
dataset.shape

(3000, 2)

In [38]:
dataset.head(10)

,class,message
emails/spam,spam,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr..."
emails/spam,spam,1) Fight The Risk of Cancer!\n\nhttp://www.adc...
emails/spam,spam,1) Fight The Risk of Cancer!\n\nhttp://www.adc...
emails/spam,spam,##############################################...
emails/spam,spam,I thought you might like these:\n\n1) Slim Dow...
emails/spam,spam,A POWERHOUSE GIFTING PROGRAM You Don't Want To...
emails/spam,spam,Help wanted. We are a 14 year old fortune 500...
emails/spam,spam,<html>\n\n<head>\n\n<title>ReliaQuote - Save U...
emails/spam,spam,TIRED OF THE BULL OUT THERE?\n\nWant To Stop L...
emails/spam,spam,"Dear ricardo1 ,\n\n\n\n<html>\n\n<body>\n\n<ce..."


In [44]:
vect = CountVectorizer()
X_train = vect.fit_transform(dataset['message'].values)

In [45]:
X_train.shape

(3000, 62964)

In [46]:
y = dataset['class'].values

In [47]:
y[0]

'spam'

In [48]:
nb = MultinomialNB()
nb.fit(X_train, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [49]:
from sklearn.model_selection import train_test_split

In [50]:
x_train, x_test, y_train, y_test = train_test_split(X_train, y, test_size=0.2)

In [51]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((2400, 62964), (600, 62964), (2400,), (600,))

In [52]:
nb.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [53]:
y_pred = nb.predict(x_test)

In [55]:
from sklearn.metrics import accuracy_score

In [56]:
accuracy_score(y_test, y_pred)

0.95833333333333337

In [60]:
inputData = ["Hey there! See you on monday", "Win free movie tickets",
            "Get 100% discount on any product and more offers"]

In [61]:
inputData = vect.transform(inputData)
nb.predict(inputData)

array(['ham', 'ham', 'spam'],
      dtype='<U4')